In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.spatial import KDTree
from bisect import insort
from collections import defaultdict

In [ ]:
def match(parents, childs, arity, thresholds=None):

    kd_tree = KDTree(data=parents, leafsize=10)

    child_to_parent = {}
    visited_child = {}
    parent_to_child = defaultdict(list)

    num_parents = len(parents) + 1 # 1-based for KDTree query

    for child_idx, child_coords in enumerate(childs):

        # Initialize child_to_parent record in dictionary
        child_to_parent[child_idx] = {}
        child_to_parent[child_idx]["path_length"] = float("inf") # The length of the shortest path
        child_to_parent[child_idx]["parent"] = None # The index of the cell to which the shortest path corresponds

        # Record coordinates in visited_child dictionary
        visited_child[child_idx] = child_coords

        # Target child_idx to lookup in KDTree
        lookup_child_idx = child_idx

        for k in range(1, num_parents):

            # Query closest parent
            dist_arr, parent_idx_arr = kd_tree.query(x=visited_child[lookup_child_idx], k=[k], workers=1)
            dist = float(dist_arr)
            parent_idx = int(parent_idx_arr)

            # Get threshold
            if thresholds:
                threshold = 2.5 * thresholds[parent_idx]
            else:
                threshold = float("inf")

            # If closest parent distance is greater than threshold, child is automatically invalidated
            if dist > threshold:
                child_to_parent[lookup_child_idx]["path_length"] = -1
                child_to_parent[lookup_child_idx]["parent"] = -1
                break

            # Add parent information to child_to_parent dictionary
            child_to_parent[lookup_child_idx]["path_length"] = dist
            child_to_parent[lookup_child_idx]["parent"] = parent_idx

            # Add child information to parent_to_child dictionary
            insort(parent_to_child[parent_idx], (dist, lookup_child_idx))

            # Check if parent has number of childs exceeding arity
            if len(parent_to_child[parent_idx]) > arity:

                # Remove the furthest child
                _, child_to_remove = parent_to_child[parent_idx].pop()

                # Case of no possible match - no more parents left
                if k == num_parents:
                    child_to_parent[child_to_remove]["path_length"] = -1
                    child_to_parent[child_to_remove]["parent"] = -1
                else: 
                # Proceed to match with next possible closest parent

                    # Reintialize child_to_parent record in dictionary
                    child_to_parent[child_to_remove]["path_length"] = float("inf")
                    child_to_parent[child_to_remove]["parent"] = None

                    # Set removed child as lookup target to match with next available neighbor
                    lookup_child_idx = child_to_remove
            else:
                # If insertion suceed, proceed to next child in list
                break

    return child_to_parent      


In [ ]:
cellprofiler_path = "/home/krarm/AutomatedCiliaMeasurements/sample_csvs"

In [ ]:
# Targeted fields to read from cellprofiler csvs
fields = ["ImageNumber", "ObjectNumber", "Location_Center_X", "Location_Center_Y", "AreaShape_MeanRadius"]

In [ ]:
nucleus_df = pd.read_csv(
    os.path.join(cellprofiler_path, "MyExpt_Nucleus.csv"), 
    skipinitialspace=True, 
    usecols=fields
)
nucleus_df.rename(columns={"ObjectNumber": "Nucleus"}, inplace=True)
nucleus_df

In [ ]:
centriole_df = pd.read_csv(
    os.path.join(cellprofiler_path, "MyExpt_Centriole.csv"), 
    skipinitialspace=True, 
    usecols=fields
)
centriole_df.rename(columns={"ObjectNumber": "Centriole"}, inplace=True)
centriole_df

In [ ]:
cilia_df = pd.read_csv(
    os.path.join(cellprofiler_path, "MyExpt_Cilia.csv"), 
    skipinitialspace=True, 
    usecols=fields
)
cilia_df.rename(columns={"ObjectNumber": "Cilia"}, inplace=True)
cilia_df

In [ ]:
grouped_nucleus = nucleus_df.groupby("ImageNumber")
grouped_centriole = centriole_df.groupby("ImageNumber")
grouped_cilia = cilia_df.groupby("ImageNumber")

In [ ]:
# Initialize c2c_df
c2c_df = pd.DataFrame(columns=['ImageNumber', 'Nucleus', 'Centriole_1', 'Centriole_2', 'Cilia', 'Nuc_Cent_1', 'Nuc_Cent_2', 'Nuc_Cil'])

# Iterate over groups 
# Note: groups in grouped_nucleus, grouped_centriole and grouped_cilia are expected to be aligned
for key in grouped_nucleus.groups.keys():
    
    # Fetch respective group
    nucleus_group = grouped_nucleus.get_group(key)
    centriole_group = grouped_centriole.get_group(key)
    cilia_group = grouped_cilia.get_group(key)

    coord_fields = ["Location_Center_X", "Location_Center_Y"]
    threshold_field = "AreaShape_MeanRadius"

    #region : Nucleus - Centriole Matching

    # Match nucleus (parent) with closest 2 centrioles (child) 
    nucleus_centriole_match_dict = match(
        parents=nucleus_group.loc[:, coord_fields].values, 
        childs=centriole_group.loc[:, coord_fields].values, 
        arity=2,
        thresholds=nucleus_group[threshold_field].to_list()
    )

    # Make df from dict and rename columns
    nucleus_centriole_match_df = pd.DataFrame.from_dict(nucleus_centriole_match_dict, orient='index')
    nucleus_centriole_match_df.rename(columns={"path_length":"Nuc_Cent", "parent":"Nucleus"}, inplace=True)
    nucleus_centriole_match_df.reset_index(inplace=True, names="Centriole") 

    # Drop unmatched/invalid centriole
    nucleus_centriole_match_df.drop(nucleus_centriole_match_df[nucleus_centriole_match_df.Nucleus == -1].index, inplace=True)   
    
    # Increment Centriole and Nucleus number since they are 1-based
    nucleus_centriole_match_df["Centriole"] += 1
    nucleus_centriole_match_df["Nucleus"] += 1

    # Sort values by nucleus number and distance from nucleus
    nucleus_centriole_match_df = nucleus_centriole_match_df.sort_values(by=['Nucleus', 'Nuc_Cent']).groupby(['Nucleus'], as_index=False).agg(list)

    # Split Centriole number and distances from nucleus
    try:
        nucleus_centriole_split_centriole_df = pd.DataFrame(nucleus_centriole_match_df['Centriole'].to_list(), columns = ['Centriole_1', 'Centriole_2'], dtype=pd.Int64Dtype())
    except ValueError:
        nucleus_centriole_split_centriole_df = pd.DataFrame(nucleus_centriole_match_df['Centriole'].to_list(), columns = ['Centriole_1'], dtype=pd.Int64Dtype())
        nucleus_centriole_split_centriole_df['Centriole_2'] = pd.NA

    try:
        nucleus_centriole_split_nc_df = pd.DataFrame(nucleus_centriole_match_df['Nuc_Cent'].to_list(), columns = ['Nuc_Cent_1', 'Nuc_Cent_2'])
    except ValueError:
        nucleus_centriole_split_nc_df = pd.DataFrame(nucleus_centriole_match_df['Nuc_Cent'].to_list(), columns = ['Nuc_Cent_1'])
        nucleus_centriole_split_nc_df['Nuc_Cent_2'] = np.nan
    nucleus_centriole_match_df = pd.concat([nucleus_centriole_match_df, nucleus_centriole_split_centriole_df, nucleus_centriole_split_nc_df], axis=1)
    nucleus_centriole_match_df.drop(['Centriole', 'Nuc_Cent'], axis=1, inplace=True)
    nucleus_centriole_match_df.drop_duplicates(inplace=True)

    #endregion

    #region : Nucleus - Cilia Matching
    
    # Match cilia (child) with closest nucleus (parent) 
    nucleus_cilia_match_dict = match(
        parents=nucleus_group.loc[:, coord_fields].values, 
        childs=cilia_group.loc[:, coord_fields].values, 
        arity=1
    )

    # Make df from dict and rename columns
    nucleus_cilia_match_df = pd.DataFrame.from_dict(nucleus_cilia_match_dict, orient='index')
    nucleus_cilia_match_df.rename(columns={"path_length":"Nuc_Cil", "parent":"Nucleus"}, inplace=True)
    nucleus_cilia_match_df.reset_index(inplace=True, names="Cilia")

    # Drop unmatched/invalid cilia
    nucleus_cilia_match_df.drop(nucleus_cilia_match_df[nucleus_cilia_match_df.Nucleus == -1].index, inplace=True)   
    
    # Increment Centriole and Nucleus number since they are 1-based
    nucleus_cilia_match_df["Cilia"] += 1
    nucleus_cilia_match_df["Nucleus"] += 1

    #endregion

    # Merge two matching dataframes
    nucleus_centriole_cilia_df = nucleus_centriole_match_df.merge(right=nucleus_cilia_match_df, how='left', on=['Nucleus'])

    # Set ImageNumber 
    nucleus_centriole_cilia_df['ImageNumber'] = key

    # Concat in c2c output
    c2c_df = pd.concat([c2c_df, nucleus_centriole_cilia_df], ignore_index=True)

c2c_df
